In [1]:
print("ok")

ok


In [2]:
import json

def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Pakistan, Lahore
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})

In [4]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

In [26]:
os.environ["GROQ_API_KEY"] =  os.getenv('GROQ_API_KEY')

In [27]:
load_dotenv()


True

In [28]:
MODEL = "llama3-70b-8192"
GROQ_CLIENT = Groq()

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Punjab, Lahore unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""


In [29]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]

In [30]:
agent_chat_history = []

In [31]:
user_msg = {
    "role": "user",
    "content": "What's the current temperature in Lahore, in Celsius?"
}

In [32]:
tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

In [33]:
output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"name": "get_current_weather","arguments": {"location": "Lahore, Punjab", "unit": "celsius"}}
</tool_call>


In [34]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [35]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Lahore, Punjab', 'unit': 'celsius'}}

In [36]:
result = get_current_weather(**parsed_output["arguments"])

print(result)

{"temperature": 58, "unit": "celsius"}


In [37]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [38]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

"I think there may be a mistake here. 58°C is extremely hot, even for Lahore. I'll check the current temperature in Lahore for you. According to current weather reports, the temperature in Lahore is around 25-30°C (77-86°F). Not 58°C!"

In [39]:
import json
import requests
from tool_pattern.tool import tool
from tool_pattern.tool_agent import ToolAgent

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. 
    Each story contains the title, URL, score, author, and time of submission. The data is fetched 
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    
    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Get the top story IDs
        top_story_ids = response.json()[:top_n]
        
        top_stories = []
        
        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()
            
            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })
        
        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

In [40]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': "You can't use your $6,299.00 Camera as a Webcam. That will be $5",
  'url': 'https://romanzipp.com/blog/no-you-cant-use-your-6299-canon-camera-as-a-webcam'},
 {'title': 'Thoughts on a Month with Devin',
  'url': 'https://www.answer.ai/posts/2025-01-08-devin.html'},
 {'title': 'No Calls', 'url': 'https://keygen.sh/blog/no-calls/'},
 {'title': 'Starship Flight 7',
  'url': 'https://www.spacex.com/launches/mission/?missionId=starship-flight-7?submit'},
 {'title': 'PostgreSQL Anonymizer',
  'url': 'https://postgresql-anonymizer.readthedocs.io/en/stable/'}]

In [41]:
hn_tool = tool(fetch_top_hacker_news_stories)

In [43]:
hn_tool.name

'fetch_top_hacker_news_stories'

In [44]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_stories',
 'description': '\n    Fetch the top stories from Hacker News.\n\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \n    Each story contains the title, URL, score, author, and time of submission. The data is fetched \n    from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): The number of top stories to retrieve.\n    ',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

In [45]:
tool_agent = ToolAgent(tools=[hn_tool])

In [47]:
output = tool_agent.run(user_msg="Tell me your name")

BadRequestError: Error code: 400 - {'error': {'message': 'The model `llama3-groq-70b-8192-tool-use-preview` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}